In [1]:
# 显示cell运行时长
%load_ext klab-autotime

In [2]:
import os
import scipy.stats as stats
import pandas as df
import numpy as np

time: 417 ms


In [3]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.layers import Embedding
from keras.layers import Input, Dense, Multiply,Flatten,RepeatVector,Permute,Dropout
from keras.layers import Conv1D,Bidirectional,LSTM
from keras.models import Model

Using TensorFlow backend.


time: 1.44 s


In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

time: 3.69 ms


In [5]:
import os
import scipy.stats as stats
import pandas as pd
import gc
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.models.word2vec import LineSentence
from sklearn.externals import joblib
from gensim.models import Doc2Vec
import pickle

time: 116 ms


In [6]:
Data_path='/home/kesci/input/bytedance/'
train_path=os.path.join(Data_path,"train_final.csv")
df_file=df.read_csv(train_path,header=None,skiprows=910000000,nrows =90000000)
df_file = reduce_mem_usage(df_file)

Mem. usage decreased to 1888.28 Mb (45.0% reduction)
time: 15min 9s


In [7]:
df_file.columns=["query_id","query","query_title_id","title","label"]

time: 971 µs


In [8]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from gensim.models import Word2Vec
import numpy as np
import gc


def train_word2vec(documents, embedding_dim):
    """
    train word2vector over traning documents
    Args:
        documents (list): list of document
        embedding_dim (int): outpu wordvector size
    Returns:
        word_vectors(dict): dict containing words and their respective vectors
    """
    model = Word2Vec(documents, workers=16,min_count=1, size=embedding_dim)
    word_vectors = model.wv
    del model
    return word_vectors


def create_embedding_matrix(tokenizer, word_vectors, embedding_dim):
    """
    Create embedding matrix containing word indexes and respective vectors from word vectors
    Args:
        tokenizer (keras.preprocessing.text.Tokenizer): keras tokenizer object containing word indexes
        word_vectors (dict): dict containing word and their respective vectors
        embedding_dim (int): dimention of word vector
    Returns:
    """
    nb_words = len(tokenizer.word_index) + 1
    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((nb_words, embedding_dim))
    print("Embedding matrix shape: %s" % str(embedding_matrix.shape))
    for word, i in word_index.items():
        try:
            embedding_vector = word_vectors[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
        except KeyError:
            print("vector not found for word - %s" % word)
    print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    return embedding_matrix


def word_embed_meta_data(documents, embedding_dim):
    """
    Load tokenizer object for given vocabs list
    Args:
        documents (list): list of document
        embedding_dim (int): embedding dimension
    Returns:
        tokenizer (keras.preprocessing.text.Tokenizer): keras tokenizer object
        embedding_matrix (dict): dict with word_index and vector mapping
    """
    documents = [x.lower().split() for x in documents]
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(documents)
    word_vector = train_word2vec(documents, embedding_dim)
    embedding_matrix = create_embedding_matrix(tokenizer, word_vector, embedding_dim)
    del word_vector
    gc.collect()
    return tokenizer, embedding_matrix


def create_train_dev_set(tokenizer, sentences_pair, is_similar, max_sequence_length, validation_split_ratio):
    """
    Create training and validation dataset
    Args:
        tokenizer (keras.preprocessing.text.Tokenizer): keras tokenizer object
        sentences_pair (list): list of tuple of sentences pairs
        is_similar (list): list containing labels if respective sentences in sentence1 and sentence2
                           are same or not (1 if same else 0)
        max_sequence_length (int): max sequence length of sentences to apply padding
        validation_split_ratio (float): contain ratio to split training data into validation data
    Returns:
        train_data_1 (list): list of input features for training set from sentences1
        train_data_2 (list): list of input features for training set from sentences2
        labels_train (np.array): array containing similarity score for training data
        leaks_train(np.array): array of training leaks features
        val_data_1 (list): list of input features for validation set from sentences1
        val_data_2 (list): list of input features for validation set from sentences1
        labels_val (np.array): array containing similarity score for validation data
        leaks_val (np.array): array of validation leaks features
    """
    sentences1 = [x[0].lower() for x in sentences_pair]
    sentences2 = [x[1].lower() for x in sentences_pair]
    train_sequences_1 = tokenizer.texts_to_sequences(sentences1)
    train_sequences_2 = tokenizer.texts_to_sequences(sentences2)
    leaks = [[len(set(x1)), len(set(x2)), len(set(x1).intersection(x2))]
             for x1, x2 in zip(train_sequences_1, train_sequences_2)]

    train_padded_data_1 = pad_sequences(train_sequences_1, maxlen=max_sequence_length)
    train_padded_data_2 = pad_sequences(train_sequences_2, maxlen=max_sequence_length)
    train_labels = np.array(is_similar)
    leaks = np.array(leaks)
    
    
    train1= open('train_feature_first.pkl', 'rb')
    train_feature = np.array(pickle.load(train1))
    train2= open('train_feature.pkl', 'rb')
    train_feature2 = np.array(pickle.load(train2))
    train_feature=np.concatenate((train_feature[10000000:],train_feature2), axis=0)
    del train_feature2
    leaks=np.concatenate((leaks,train_feature), axis=1)
    del train_feature
    gc.collect()

    shuffle_indices = np.random.permutation(np.arange(len(train_labels)))
    train_data_1_shuffled = train_padded_data_1[shuffle_indices]
    train_data_2_shuffled = train_padded_data_2[shuffle_indices]
    train_labels_shuffled = train_labels[shuffle_indices]
    leaks_shuffled = leaks[shuffle_indices]

    dev_idx = max(1, int(len(train_labels_shuffled) * validation_split_ratio))

    del train_padded_data_1
    del train_padded_data_2
    gc.collect()

    train_data_1, val_data_1 = train_data_1_shuffled[:-dev_idx], train_data_1_shuffled[-dev_idx:]
    train_data_2, val_data_2 = train_data_2_shuffled[:-dev_idx], train_data_2_shuffled[-dev_idx:]
    labels_train, labels_val = train_labels_shuffled[:-dev_idx], train_labels_shuffled[-dev_idx:]
    leaks_train, leaks_val = leaks_shuffled[:-dev_idx], leaks_shuffled[-dev_idx:]

    return train_data_1, train_data_2, labels_train, leaks_train, val_data_1, val_data_2, labels_val, leaks_val


def create_test_data(tokenizer, test_sentences_pair, max_sequence_length):
    """
    Create training and validation dataset
    Args:
        tokenizer (keras.preprocessing.text.Tokenizer): keras tokenizer object
        test_sentences_pair (list): list of tuple of sentences pairs
        max_sequence_length (int): max sequence length of sentences to apply padding
    Returns:
        test_data_1 (list): list of input features for training set from sentences1
        test_data_2 (list): list of input features for training set from sentences2
    """
    test_sentences1 = [x[0].lower() for x in test_sentences_pair]
    test_sentences2 = [x[1].lower() for x in test_sentences_pair]

    test_sequences_1 = tokenizer.texts_to_sequences(test_sentences1)
    test_sequences_2 = tokenizer.texts_to_sequences(test_sentences2)
    leaks_test = [[len(set(x1)), len(set(x2)), len(set(x1).intersection(x2))]
                  for x1, x2 in zip(test_sequences_1, test_sequences_2)]

    leaks_test = np.array(leaks_test)
    test_data_1 = pad_sequences(test_sequences_1, maxlen=max_sequence_length)
    test_data_2 = pad_sequences(test_sequences_2, maxlen=max_sequence_length)

    return test_data_1, test_data_2, leaks_test

time: 8.43 ms


In [9]:
sentences1 = list(df_file['query'])
sentences2 = list(df_file['title'])
is_similar = list(df_file['label'])


####################################
######## Word Embedding ############
####################################

words_1 = list(set(list(df_file['query'])))
#words_1 = list(set(list(df.concat([df_file['query'],test_file['query']],axis=0))))
#words_2  = list(df.concat([df_file['title'],test_file['title']],axis=0))
# creating word embedding meta data for word embedding 
tokenizer, embedding_matrix = word_embed_meta_data(words_1,200) #+ words_2,  100)
print(len(embedding_matrix))
embedding_meta_data = {
	'tokenizer': tokenizer,
	'embedding_matrix': embedding_matrix
}

## creating sentence pairs
sentences_pair = [(x1, x2) for x1, x2 in zip(sentences1, sentences2)]
del sentences1
del sentences2
del words_1
#del words_2

Embedding matrix shape: (511560, 200)
Null word embeddings: 1
511560
time: 6min 29s


In [10]:
EMBEDDING_DIM = 200

MAX_SEQUENCE_LENGTH = 20
VALIDATION_SPLIT = 0.1

RATE_DROP_DENSE = 0.2
NUMBER_LSTM = 6
NUMBER_DENSE_UNITS = 6
ACTIVATION_FUNCTION = 'relu'

time: 1.34 ms


In [11]:
train_data_x1, train_data_x2, train_labels, leaks_train, \
        val_data_x1, val_data_x2, val_labels, leaks_val = create_train_dev_set(tokenizer, sentences_pair,
                                                                               is_similar, MAX_SEQUENCE_LENGTH,
                                                                               VALIDATION_SPLIT)

time: 1h 4min 6s


In [12]:
if train_data_x1 is None:
    print("++++ !! Failure: Unable to train model ++++")
nb_words = len(tokenizer.word_index) + 1

time: 620 µs


In [13]:
from keras.layers import  GlobalMaxPooling1D, Concatenate
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization

time: 909 µs


In [14]:
embedding_layer = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],
                                    input_length=MAX_SEQUENCE_LENGTH, trainable=True)
input_query = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
input_title = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_query = embedding_layer(input_query)
embedded_title = embedding_layer(input_title)
convs_query = []
convs_title = []
for kernel_size in[3,4,5]:
    c_query = Conv1D(64, kernel_size, activation='relu')(embedded_query)
    c_query = GlobalMaxPooling1D()(c_query)
    c_title = Conv1D(32, kernel_size, activation='relu')(embedded_title)
    c_title = GlobalMaxPooling1D()(c_title)
    convs_query.append(c_query)
    convs_title.append(c_title)

convs_query.extend(convs_title)
leaks_input = Input(shape=(leaks_train.shape[1],))
leaks_dense = Dense(int(NUMBER_DENSE_UNITS/2), activation=ACTIVATION_FUNCTION)(leaks_input)
convs_query.append(leaks_dense)
merged = concatenate(convs_query)
merged = BatchNormalization()(merged)
dropout_x = Dropout(RATE_DROP_DENSE)(merged)
merged = Dense(NUMBER_DENSE_UNITS, activation=ACTIVATION_FUNCTION)(merged)
merged = BatchNormalization()(merged)
merged = Dropout(RATE_DROP_DENSE)(merged)
preds = Dense(1, activation='sigmoid')(merged)
model = Model(inputs=[input_query, input_title, leaks_input], outputs=preds)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 200)      102312000   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 18, 64)       38464       embedding_1[0][0]                
__________

In [15]:
from keras.optimizers import Nadam 
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

time: 46.5 ms


In [16]:
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping, ModelCheckpoint
import time
early_stopping = EarlyStopping(monitor='val_loss', patience=1)
checkpoint_dir = './checkpoints/' + str(int(time.time())) + '/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
bst_model_path = checkpoint_dir + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=False)
tensorboard = TensorBoard(log_dir=checkpoint_dir + "logs/{}".format(time.time()))

time: 5.65 s


In [17]:
model.fit([train_data_x1, train_data_x2, leaks_train], train_labels,
                  validation_data=([val_data_x1, val_data_x2, leaks_val], val_labels),
                  epochs=10, batch_size=20480, shuffle=True,verbose=1,callbacks=[early_stopping, model_checkpoint, tensorboard])

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:109: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 102312000 elements. This may consume a large amount of memory.
  num_elements)


Train on 81000000 samples, validate on 9000000 samples
Epoch 1/10
81000000/81000000 [==============================] - 3316s 41us/step - loss: 0.4667 - acc: 0.8138 - val_loss: 0.4543 - val_acc: 0.8179
Epoch 2/10
81000000/81000000 [==============================] - 3310s 41us/step - loss: 0.4514 - acc: 0.8180 - val_loss: 0.4514 - val_acc: 0.8181
Epoch 3/10
81000000/81000000 [==============================] - 3309s 41us/step - loss: 0.4460 - acc: 0.8183 - val_loss: 0.4508 - val_acc: 0.8182
Epoch 4/10
81000000/81000000 [==============================] - 3310s 41us/step - loss: 0.4401 - acc: 0.8190 - val_loss: 0.4506 - val_acc: 0.8182
Epoch 5/10
81000000/81000000 [==============================] - 3312s 41us/step - loss: 0.4334 - acc: 0.8202 - val_loss: 0.4514 - val_acc: 0.8175


time: 4h 36min 32s


In [18]:
del train_data_x1
del train_data_x2
del leaks_train
del val_data_x1
del val_data_x2
del leaks_val

time: 606 µs


In [19]:
print(checkpoint_dir)

./checkpoints/1565246039/
time: 1.22 ms


In [20]:
test_path=os.path.join(Data_path,"test_final_part1.csv")
test_file=df.read_csv(test_path,header=None)
test_file = reduce_mem_usage(test_file)
test_file.columns=["query_id","query","query_title_id","title"]
test_sentences_pair = [(x1, x2) for x1, x2 in zip(test_file['query'], test_file['title'])]

Mem. usage decreased to 400.54 Mb (34.4% reduction)
time: 44.2 s


In [21]:
test_data_x1, test_data_x2, leaks_test = create_test_data(tokenizer,test_sentences_pair,MAX_SEQUENCE_LENGTH)

time: 13min 19s


In [22]:
test= open('test_feature.pkl', 'rb')
test_feature = np.array(pickle.load(test))
leaks_test=np.concatenate((leaks_test,test_feature), axis=1)

time: 5.9 s


In [23]:
from keras.models import load_model
model = load_model(bst_model_path)
preds = list(model.predict([test_data_x1, test_data_x2, leaks_test], verbose=1).ravel())

20000000/20000000 [==============================] - 1168s 58us/step
time: 19min 38s


In [24]:
sub=test_file[['query_id','query_title_id']]
sub['prediction'] = preds

time: 4.64 s


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
sub.to_csv('textcnn.csv',index=False,header=False)

time: 1min 29s


In [26]:
!https_proxy="http://klab-external-proxy" ./kesci_submit -file textcnn.csv -token 9dd65ea29b2ce893

Kesci Submit Tool 3.0

> 已验证Token
> 提交文件 textcnn.csv (575561.33 KiB)
> 文件已上传        
> 提交完成
time: 24.9 s
